In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
from tensorflow.python.framework import ops
ops.disable_eager_execution()

import sys 
import yaml
import math
import time
import random
import traceback
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import scipy.sparse
from scipy.ndimage import gaussian_filter

from tqdm.auto import tqdm

import numpy.fft as FFT
from typing import List, Dict

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from tensorflow.keras.layers import (Input, Conv2D, Dense, Flatten, 
                                     MaxPooling2D, RepeatVector, Lambda,
                                     LeakyReLU, Dropout, Add, Activation, AveragePooling2D,
                                    ZeroPadding2D, BatchNormalization)
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.backend as K

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

from holodecml.library.losses import SymmetricCrossEntropy
from holodecml.library.callbacks import get_callbacks
from holodecml.library.FourierOpticsLib import OpticsFFT, OpticsIFFT

from multiprocessing import cpu_count, Pool

Using TensorFlow backend.


### Load the configuration

In [2]:
config_file = "config.yml"

In [3]:
with open(config_file) as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

In [4]:
config["path_save"] = "resnet"

try:
    os.makedirs(config["path_save"])
except:
    pass

### Set up the logger

In [5]:
root = logging.getLogger()
root.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

# Stream output to stdout
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
root.addHandler(ch)
#########

# Save the log file
fp = os.path.join(f'{config["path_save"]}/log.txt')
fh = logging.FileHandler(fp,
                         mode='w',
                         encoding='utf-8')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
root.addHandler(fh)

### Set up some globals

In [6]:
path_data = config["path_data"]
num_particles = config["num_particles"]
split = 'train'
subset = False
output_cols = ["x", "y", "z", "d", "hid"]

batch_size = config["conv2d_network"]["batch_size"]

input_shape = (600, 400, 4)

n_particles = config["num_particles"]
output_channels = len(output_cols) - 1

In [7]:
num_particles_dict = {
    1: '1particle',
    3: '3particle',
    'multi': 'multiparticle',
    '50-100': '50-100'}

split_dict = {
    'train' : 'training',
    'test'   : 'test',
    'valid': 'validation'}


class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(
        
        self, 
        path_data: str, 
        num_particles: int, 
        split: str, 
        subset: bool, 
        output_cols: List[str], 
        batch_size: int, 
        shuffle: bool = True,
        maxnum_particles: int = False,
        scaler: Dict[str, str] = False) -> None:
        
        'Initialization'
        self.ds = self.open_dataset(path_data, num_particles, split)
        self.batch_size = batch_size
        self.output_cols = [x for x in output_cols if x != 'hid']        
        self.subset = subset
        self.hologram_numbers = self.ds.hologram_number.values
        if shuffle:
            random.shuffle(self.hologram_numbers)
        self.num_particles = num_particles
        self.xsize = len(self.ds.xsize.values)
        self.ysize = len(self.ds.ysize.values)
        self.shuffle = shuffle
        self.maxnum_particles = maxnum_particles
                
        if not scaler:
            self.scaler = {col: StandardScaler() for col in output_cols}
            for col in output_cols:
                scale = self.ds[col].values
                self.scaler[col].fit(scale.reshape(scale.shape[-1], -1))
        else:
            self.scaler = scaler
        
    def get_transform(self):
        return self.scaler

    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(len(self.hologram_numbers) / self.batch_size)
    
    def __getitem__(self, idx):
        'Generate one batch of data'
        holograms = self.hologram_numbers[
            idx * self.batch_size: (idx + 1) * self.batch_size
        ]
        x_out, y_out, w_out = self._batch(holograms)
        return x_out, y_out, w_out
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            random.shuffle(self.hologram_numbers)
            
    def standardize(self, X):
        X = (X-np.mean(X)) / (np.std(X))
        return X
    
    def reshape(self, X):
        x, y = X.shape
        return X.reshape((x,y,1))
            
    def _batch(self, holograms: List[int]):
        'Create a batch of data'
        try:
        
            x_out = np.zeros((
                len(holograms), self.xsize, self.ysize, 4
            ))
            y_out = np.zeros((
                len(holograms), 
                self.maxnum_particles if self.maxnum_particles else self.num_particles, 
                len(self.output_cols)
            ))
            # Move the scaler.transform to here
            
            a = time.time()
            for k, hologram in enumerate(holograms):
                im = self.ds["image"][hologram].values
                A = self.standardize(im)
                F = OpticsFFT(A)                     
                R = self.reshape(self.standardize(np.log(np.abs(F))))
                P = self.reshape(self.standardize(np.real(F)))
                Q = self.reshape(self.standardize(np.imag(F)))
                x_out[k] = np.concatenate((self.reshape(A), R, P, Q), axis = 2)
                particles = np.where(self.ds["hid"] == hologram + 1)[0]  
                for l, p in enumerate(particles):
                    for m, col in enumerate(self.output_cols):
                        val = self.ds[col][p].values
                        y_out[k, l, m] = self.scaler[col].transform(
                            val.reshape(1, -1)
                        )
                if self.maxnum_particles and len(particles) < self.maxnum_particles:
                    for l in range(len(particles), self.maxnum_particles):
                        for m, col in enumerate(self.output_cols):
                            val = y_out[k, l, m]
                            y_out[k, l, m] = self.scaler[col].transform(
                                val.reshape(1, -1)
                            )
            #
            # convert y_out to sparse if we are using padding
#             if self.maxnum_particles:
#                 y_out = sparse_vstack([
#                     csr_matrix(y_out[i]) for i in y_out.shape[0]
#                 ])
            
            #x_out = np.expand_dims(x_out, axis=-1)
            return x_out, y_out, [None] #class weights option
        
        except:
            print(traceback.print_exc())
    
    def open_dataset(self, path_data, num_particles, split):
        """
        Opens a HOLODEC file

        Args: 
            path_data: (str) Path to dataset directory
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'

        Returns:
            ds: (xarray Dataset) Opened dataset
        """
        path_data = os.path.join(path_data, self.dataset_name(num_particles, split))

        if not os.path.isfile(path_data):
            print(f"Data file does not exist at {path_data}. Exiting.")
            raise 

        ds = xr.open_dataset(path_data)
        return ds
    
    def dataset_name(self, num_particles, split, file_extension='nc'):
        """
        Return the dataset filename given user inputs

        Args: 
            num_particles: (int or str) Number of particles per hologram
            split: (str) Dataset split of either 'train', 'valid', or 'test'
            file_extension: (str) Dataset file extension

        Returns:
            ds_name: (str) Dataset name
        """

        valid = [1,3,'multi','50-100']
        if num_particles not in valid:
            raise ValueError("results: num_particles must be one of %r." % valid)
        num_particles = num_particles_dict[num_particles]

        valid = ['train','test','valid']
        if split not in valid:
            raise ValueError("results: split must be one of %r." % valid)
        split = split_dict[split]
        ds_name = f'synthetic_holograms_{num_particles}_{split}.{file_extension}'

        return ds_name

In [8]:
train_gen = DataGenerator(
    path_data, num_particles, "train", subset, 
    output_cols, batch_size, maxnum_particles = 3
)

In [9]:
x, y, z = train_gen.__getitem__(0)

In [10]:
# scaler = {col: StandardScaler() for col in train_gen.output_cols}
# for col in train_gen.output_cols:
#     scale = train_gen.ds[col].values
#     scaler[col].fit(scale.reshape(scale.shape[-1], -1))
#     result = scaler[col].transform(scale.reshape(scale.shape[-1], -1))
#     print(col, min(result), max(result), np.mean(result), np.std(result))

In [11]:
train_scalers = train_gen.get_transform()

In [12]:
valid_gen = DataGenerator(
    path_data, num_particles, "test", subset, 
    output_cols, batch_size, scaler = train_scalers, maxnum_particles = 3
)

### Initialize callbacks

In [13]:
callbacks = get_callbacks(config)

### Set up a model

##### Custom losses

In [14]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res/(SS_tot + K.epsilon())

def keras_mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

In [15]:
# def wsme(y_true, y_pred):
#     cond1 = K.equal(y_true, 0.0)
#     zero = K.switch(cond1,K.square(y_pred - y_true), 
#                              K.zeros_like(y_true))
#     cond2 = K.greater(y_true, 0.0)
#     real = K.switch(cond2, 
#                              K.square(y_pred - y_true), 
#                              K.zeros_like(y_true))
#     w1 = K.sum(K.cast(cond1, "float"))
#     w2 = K.sum(K.cast(cond2, "float"))
#     total = w1 + w2
#     zero = K.sum(zero) / w1
#     real = K.sum(real) / w2
#     return (real + zero)

def wsme(y_true, y_pred):
    
    w = K.abs(K.mean(y_true[1]))
    w = w / (1 - w)
    
    # w = K.sum(K.cast(K.greater(y_true[1], 0), "float")) # Number actually not zero
    
    error = K.square(y_true - y_pred)
    error = K.switch(K.equal(y_true, 0), w * error, error)
    return error

In [16]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [17]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s=2):
    """
  

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    return X

In [18]:
def ResNet50(input_shape=input_shape):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((1, 1))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    #X = Dense(10, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
    #X = Dropout(0.1)(X)
    #X = Dense(1, activation='linear', kernel_initializer=glorot_uniform(seed=0))(X)
   

    # Create model
    #model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return X_input, X #model

In [19]:
from keras.initializers import glorot_uniform

n_particles = 3

custom_losses = {
    "sce": SymmetricCrossEntropy(0.5, 0.5),
    "weighted_mse": wsme,
    "r2": R2,
    "rmse": rmse
}

class Conv2DNeuralNetwork(object):
    """
    A Conv2D Neural Network Model that can support an arbitrary numbers of
    layers.

    Attributes:
        filters: List of number of filters in each Conv2D layer
        kernel_sizes: List of kernel sizes in each Conv2D layer
        conv2d_activation: Type of activation function for conv2d layers
        pool_sizes: List of Max Pool sizes
        dense_sizes: Sizes of dense layers
        dense_activation: Type of activation function for dense layers
        output_activation: Type of activation function for output layer
        lr: Optimizer learning rate
        optimizer: Name of optimizer or optimizer object.
        adam_beta_1: Exponential decay rate for the first moment estimates
        adam_beta_2: Exponential decay rate for the first moment estimates
        sgd_momentum: Stochastic Gradient Descent momentum
        decay: Optimizer decay
        loss: Name of loss function or loss object
        batch_size: Number of examples per batch
        epochs: Number of epochs to train
        verbose: Level of detail to provide during training
        model: Keras Model object
    """
    def __init__(
        self, 
        filters=(8,), 
        kernel_sizes=(5,),
        conv2d_activation="relu", 
        pool_sizes=(4,), 
        pool_dropout=0.0,
        dense_sizes=(64,),
        dense_activation="relu", 
        dense_dropout = 0.0,
        output_activation="linear",
        lr=0.001, 
        optimizer="adam", 
        adam_beta_1=0.9,
        adam_beta_2=0.999, 
        sgd_momentum=0.9, 
        decay=0, 
        loss="mse",
        metrics = [], 
        batch_size=32, 
        epochs=2, 
        verbose=0):
        
        self.filters = filters
        self.kernel_sizes = [tuple((v,v)) for v in kernel_sizes]
        self.conv2d_activation = conv2d_activation
        self.pool_sizes = [tuple((v,v)) for v in pool_sizes]
        self.pool_dropout = pool_dropout
        self.dense_sizes = dense_sizes
        self.dense_activation = dense_activation
        self.dense_dropout = dense_dropout
        self.output_activation = output_activation
        self.lr = lr
        self.optimizer = optimizer
        self.optimizer_obj = None
        self.adam_beta_1 = adam_beta_1
        self.adam_beta_2 = adam_beta_2
        self.sgd_momentum = sgd_momentum
        self.decay = decay
        self.loss = custom_losses[loss] if loss in custom_losses else loss
        self.metrics = []
        for m in metrics:
            if m in custom_losses:
                self.metrics.append(custom_losses[m])
            else:
                self.metrics.append(m)
        self.batch_size = batch_size
        self.epochs = epochs
        self.verbose = verbose
        self.model = None
        
        if self.conv2d_activation == "leakyrelu":
            self.conv2d_activation = LeakyReLU(alpha=0.1)
        if self.dense_activation == "leakyrelu":
            self.dense_activation = LeakyReLU(alpha=0.1)
        if self.output_activation == "leakyrelu":
            self.output_activation = LeakyReLU(alpha=0.1)

    def build_neural_network(self, input_shape, n_particles, output_shape):
        """Create Keras neural network model and compile it."""
                
        # Input
        conv_input, nn_model = ResNet50(input_shape)
        nn_model = RepeatVector(n_particles, name = "repeat")(nn_model)
        nn_model = Dense(output_shape,
                         activation=self.output_activation,
                         name=f"dense_output")(nn_model)
        nn_model = Lambda(
            self.LastLayer,
            input_shape = (n_particles, output_shape),
            name="coordinate"
        )(nn_model)
        
        self.model = Model(
            inputs = conv_input, 
            outputs = nn_model
        )
        
        if self.optimizer == "adam":
            self.optimizer_obj = Adam(lr=self.lr, clipnorm = 1.0)
        elif self.optimizer == "sgd":
            self.optimizer_obj = SGD(lr=self.lr, momentum=self.sgd_momentum,
                                     decay=self.decay)
            
        lr_metric = get_lr_metric(self.optimizer_obj)
        self.model.compile(
            optimizer=self.optimizer_obj, 
            loss=self.loss,
            metrics=self.metrics + [lr_metric]
        )
        self.model.summary()
    
    def LastLayer(self, x):
        return 1.75 * K.tanh(x / 100) 

    def predict(self, x):
        y_out = self.model.predict(np.expand_dims(x, axis=-1),
                                   batch_size=self.batch_size)
        return y_out

    def predict_proba(self, x):
        y_prob = self.model.predict(x, batch_size=self.batch_size)
        return y_prob
    
    def load_weights(self, weights):
        try:
            self.model.load_weights(weights)
            self.model.compile(
                optimizer=self.optimizer, 
                loss=self.loss, 
                metrics=self.metrics
            )
        except:
            print("You must first call build_neural_network before loading weights. Exiting.")
            sys.exit(1)



In [21]:
K.clear_session()
ops.reset_default_graph()
mod = Conv2DNeuralNetwork(**config["conv2d_network"])

In [22]:
mod.build_neural_network(input_shape, n_particles, output_channels)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 400, 4) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 602, 402, 4)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 298, 198, 64) 12608       zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 298, 198, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

### Train a model

In [ ]:
mod.model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    epochs=config["conv2d_network"]["epochs"],
    verbose=True,
    callbacks=callbacks,
    use_multiprocessing=True,
    workers=16,
    max_queue_size=32
)

Epoch 1/100
290/782 [==========>...................] - ETA: 12:22 - loss: 1.0793 - R2: 0.3456 - lr: 2.5000e-04

### Use sklearn to do Bayesian optimzation of hyperparameters

##### (Based on https://medium.com/@crawftv/parameter-hyperparameter-tuning-with-bayesian-optimization-7acf42d348e1)

In [ ]:
# import gc
# import skopt
# from skopt import gbrt_minimize, gp_minimize
# from skopt.utils import use_named_args
# from skopt.space import Real, Categorical, Integer  

In [ ]:
# filter1 = Integer(low=1, high=64, name='filter1')
# filter2 = Integer(low=1, high=64, name='filter2')
# filter3 = Integer(low=1, high=64, name='filter3')
# kernel_sizes = Integer(low=1, high=10, name='kernel_sizes')
# pool_sizes = Integer(low=1, high=50, name='pool_sizes')
# dense_1 = Integer(low=10, high=1000, name='dense_1')
# dense_2 = Integer(low=10, high=1000, name='dense_2')
# learning_rate = Real(low=1e-5, high=1e-2, prior='log-uniform',
#                          name='learning_rate')

# # To add 
# ## kernel size dimensions 
# ## pool size dimensions 
# ## dropout for pool layers
# ## dropout for dense layers
# ## Number of dense layers
# ## scaling factor I use in Lambda function

# dimensions = [
#     filter1,
#     filter2,
#     filter3,
#     kernel_sizes,
#     pool_sizes,
#     dense_1,
#     dense_2,
#     learning_rate
# ]

# default_parameters = [52, 64, 41, 2, 29, 175, 912, 0.01]

# #[57, 57, 18, 6, 10, 781, 607, 0.005285282058684279]
# # [8, 12, 16, 5, 5, 64, 32, 1e-3]

In [ ]:
# @use_named_args(dimensions=dimensions)
# def fitness(
#     filter1, filter2, filter3,
#     kernel_sizes, pool_sizes,
#     dense_1, dense_2, learning_rate):
    
    
#     a = time.time()
    
#     # Update the configuration
#     c = config["conv2d_network"]
#     c["filters"] = [filter1, filter2, filter3]
#     c["kernel_sizes"] = [kernel_sizes, kernel_sizes, kernel_sizes]
#     c["pool_sizes"] = [pool_sizes, pool_sizes, pool_sizes]
#     c["dense_sizes"] = [dense_1, dense_2]
#     c["lr"] = learning_rate
#     c["epochs"] = 1
    
#     save_path = os.path.join(config["path_save"], "log.txt")
#     with open(save_path, "a+") as fid:
#         fid.write("------------------------------------\n")
#         fid.write("Starting run\n")
#         fid.write(f"filters: {filter1,filter2,filter3} kernel_size: {kernel_sizes} pool_size: {pool_sizes} dense1: {dense_1} dense2: {dense_2} lr: {learning_rate}\n")

#     # Load the model
#     model = Conv2DNeuralNetwork(**c)
#     model.build_neural_network(input_shape, n_particles, output_channels)
    
#     # Load callbacks, though we prob. wont need them for 1 epoch optimization
#     callbacks = get_callbacks(config)
    
#     # Train a model
#     blackbox = model.model.fit(
#         train_gen,
#         validation_data=valid_gen,
#         epochs=config["conv2d_network"]["epochs"],
#         verbose=True,
#         callbacks=callbacks,
#         use_multiprocessing=True,
#         workers=24,
#         max_queue_size=100
#     )
    
#     # Return the validation accuracy for the last epoch.
#     objective = blackbox.history['val_R2'][-1]
    
#     with open(save_path, "a+") as fid:
#         fid.write(f"Final result: {objective}\n")
#         fid.write(f"This iteration took {time.time() - a} s\n")

#     # Delete the Keras model with these hyper-parameters from memory.
#     del model
    
#     # Garbage collection
#     gc.collect()
    
#     # Clear the Keras session, otherwise it will keep adding new
#     # models to the same TensorFlow graph each time we create
#     # a model with a different set of hyper-parameters.
#     K.clear_session()
#     #tf.compat.v1.reset_default_graph()
#     ops.reset_default_graph()
    
#     # The optimizer aims for the lowest score
#     # For categorical problems, return the negative accuracy
#     return objective

In [ ]:
# # Multi-GPU multiprocessing potentially 
# # https://github.com/scikit-optimize/scikit-optimize/issues/737

# gp_result = gp_minimize(
#     func=fitness,
#     dimensions=dimensions,
#     n_calls=50,
#     noise=0.01,
#     n_jobs=-1,
#     kappa = 5,
#     x0=default_parameters
# )